# Dataset

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

In [3]:
required_libs = [ ("numpy", "numpy"),
                 ("pandas", "pandas"),
                 ("seaborn", "seaborn"),
                 ("matplotlib", "matplotlib"),
                 ("sklearn", "sklearn"),
                 ("colorama", "colorama"),
                 ("emoji", "emoji")
                ]
                 #("pycaret", "pycaret")

In [4]:
def is_lib_exists(name):
    import importlib
    lib = importlib.util.find_spec(name)
    return lib is not None

In [5]:
for (clz,lib) in required_libs:
    if not is_lib_exists(clz):
        print(f"Installing {lib}")
        !pip install {lib}
    else:
        print(f"{lib} exists")

numpy exists
pandas exists
seaborn exists
matplotlib exists
sklearn exists
colorama exists
emoji exists


In [6]:
COLAB = True

DATASET_NAME = '4D.zip'

FEATURE_DATASET_PREFIX = 'feature_matrix_d2_v3'

In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
import traceback
import catboost
import gc
from pathlib import Path
from dateutil.relativedelta import *
from datetime import *
from catboost import *
from catboost import datasets
from catboost import CatBoostClassifier
from scipy import stats
from scipy.stats.stats import pearsonr
np.set_printoptions(precision=4)
pd.options.display.max_columns = None

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [8]:
%matplotlib inline
%aimport

Modules to reload:
all-except-skipped

Modules to skip:



In [9]:
from IPython.display import display

In [10]:
# check catboost version
print(catboost.__version__)
!python --version

0.24.1
Python 3.6.9


In [11]:
# colab setup
if COLAB:
  !rm -rf dl-projects
  !git clone https://github.com/mengwangk/dl-projects
  
  !cp dl-projects/utils* .
  !cp dl-projects/preprocess* .
  !cp dl-projects/plot* .
  
  from google.colab import drive
  drive.mount('/content/gdrive')
  GDRIVE_DATASET_FOLDER = Path('gdrive/My Drive/datasets/')
  DATASET_PATH = GDRIVE_DATASET_FOLDER
  ORIGIN_DATASET_PATH = Path('dl-projects/datasets')
  !ls -l gdrive/"My Drive"/datasets/ --block-size=M

DATASET = DATASET_PATH/f"{FEATURE_DATASET_PREFIX}.ft"
ORIGIN_DATASET = ORIGIN_DATASET_PATH/DATASET_NAME

Cloning into 'dl-projects'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 2228 (delta 9), reused 6 (delta 2), pack-reused 2210
Receiving objects: 100% (2228/2228), 80.11 MiB | 11.29 MiB/s, done.
Resolving deltas: 100% (1388/1388), done.
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
total 4948M
-rw------- 1 root root   17M Jul  6 14:06 feature_matrix_2020_apr.ft
-rw------- 1 root root   17M Jul  6 14:06 feature_matrix_2020_apr_orig.pkl
-rw------- 1 root root   17M Mar  1  2020 feature_matrix_2020_feb.ft
-rw------- 1 root root   17M Mar  1  2020 feature_matrix_2020_jan.ft
-rw------- 1 root root   17M Jul  7 13:52 feature_matrix_2020_jul.ft
-rw------- 1 root root   17M Jul  7 13:52 feature_matrix_2020_jul_orig.pkl
-rw------- 1 root root   17M Jul  5 07:30 feature_matrix_2020_jun.ft
-rw------- 1 root roo

## EDA

In [12]:
from preprocess import *
from utils import feature_selection, plot_feature_importances
from plot import plot_correlation_matrix, plot_labeled_scatter

In [ ]:
data = pd.read_feather(DATASET)
origin_data = format_tabular(ORIGIN_DATASET)

In [ ]:
jan_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_jan.ft")
feb_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_feb.ft")
mar_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_mar.ft")
apr_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_apr.ft")
may_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_may.ft")
jun_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_jun.ft")
jul_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_jul.ft")

In [ ]:
new_data = data.append(jan_2020[data.columns],ignore_index=True)
new_data = new_data.append(feb_2020[data.columns],ignore_index=True)
new_data = new_data.append(mar_2020[data.columns],ignore_index=True)
new_data = new_data.append(apr_2020[data.columns],ignore_index=True)
new_data = new_data.append(may_2020[data.columns],ignore_index=True)
new_data = new_data.append(jun_2020[data.columns],ignore_index=True)
new_data = new_data.append(jul_2020[data.columns],ignore_index=True)
data.shape, new_data.shape 

In [ ]:
data = new_data

In [ ]:
data.isna().sum().sort_values(ascending=False)

In [ ]:
data.info(max_cols=500, null_counts=True)
#data.columns.tolist()

In [ ]:
feature_matrix = data
feature_matrix = data.fillna(0)
feature_matrix.sort_values(by=['time', 'MAX(Results.LuckyNo)'], inplace=True)
feature_matrix.head(20)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, chi2, f_regression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold

other_features = ['Label', 'NumberId', 'time', 'TotalStrike',  'month', 'year']
#feature_matrix = feature_selection(data)

def select_features(df, func=f_regression):
  X = df.drop(columns=other_features)
  y = df.Label
  fs = SelectKBest(score_func=func, k=50)
  X_selected = fs.fit_transform(X, y)
  mask = fs.get_support()
  return X.columns[mask]

def reduce_features(df, lower_threshold=10, upper_threshold=10000):
  cols_to_drop = []
  for col in df.columns:
    if df[col].nunique() > upper_threshold or df[col].nunique() < lower_threshold:
      cols_to_drop.append(col)
  print(cols_to_drop)
  return cols_to_drop, df.drop(columns=cols_to_drop)

def select_by_variance(df, threshold=0):
  """Select features by variance"""
  filter = VarianceThreshold(threshold=threshold)
  filter.fit(df)
  cols_to_drop = [column for column in df.columns
                  if column not in df.columns[filter.get_support()]]
  print(f"No of columns to drop - {len(cols_to_drop)}")
  print(cols_to_drop)
  return cols_to_drop

def select_duplicates(df):
  df_transposed = df.head(20000).T
  print(df_transposed.duplicated().sum())
  unique_features = df_transposed.drop_duplicates(keep='first').T
  print(unique_features.shape)
  duplicated_features = [dup_col for dup_col in df.columns if dup_col not in unique_features.columns]
  return duplicated_features

def select_correlated(df):
  correlated_features = set()
  correlation_matrix = df.corr()
  for i in range(len(correlation_matrix .columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)
  print(len(correlated_features))
  print(correlated_features)
  return correlated_features


In [ ]:
selected_features = feature_matrix.columns.to_list()
display(len(selected_features), selected_features)

In [ ]:
df_selected_features = feature_matrix[selected_features]

In [ ]:
def split_data(df, dt):
  y_train = df.loc[df['time'] < dt, 'Label']
  y_validation = df.loc[df['time'] == dt, 'Label']

  X_train = df[df['time'] < dt].drop(columns = ['NumberId', 'time', 'Label', 'TotalStrike', 'year'])
  X_validation = df[df['time'] == dt].drop(columns = ['NumberId', 'time', 'Label', 'TotalStrike', 'year'])

  # Reduce by standard deviation
  #X_train = X_train.loc[:, X_train.std() > .0]
  #X_validation = X_validation[X_train.columns]

  return X_train, X_validation, y_train, y_validation
 

In [ ]:
TARGET_MONTH = datetime(2020,1,1)

X_train, X_validation, y_train, y_validation = split_data(df_selected_features, TARGET_MONTH)
X_train.shape, X_validation.shape, y_train.shape, y_validation.shape

In [ ]:
cols_to_drop = select_by_variance(X_train,0.0)
X_train = X_train.drop(columns=cols_to_drop)
X_validation = X_validation[X_train.columns]
print(X_train.shape, X_validation.shape)

cols_to_drop = select_by_variance(X_train,0.01)
X_train = X_train.drop(columns=cols_to_drop)
X_validation = X_validation[X_train.columns]
print(X_train.shape, X_validation.shape)

cols_to_drop = select_duplicates(X_train)
X_train = X_train.drop(columns=cols_to_drop)
X_validation = X_validation[X_train.columns]
print(X_train.shape, X_validation.shape)

cols_to_drop = select_correlated(X_train)
X_train = X_train.drop(columns=cols_to_drop)
X_validation = X_validation[X_train.columns]
print(X_train.shape, X_validation.shape)

In [ ]:
from pycaret.utils import version
print(version())

In [ ]:
y_train

In [ ]:
train_label = pd.DataFrame({"Label": y_train })
train_label.info()

In [ ]:
train_data = X_train
train_data['Label'] = train_label
train_data.info()

In [ ]:
train_data.isna().sum().sort_values(ascending=False)
train_data.head(10)

In [ ]:
from pycaret.classification import *
clf1 = setup(train_data, target = 'Label', session_id=123, log_experiment=True, experiment_name='lotto')

In [ ]:
best_model = compare_models()